In [1]:
library(splatter)
library(tidyverse)


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

In [2]:
# Rewritten
splatSimBatchEffects_multi <- function(sim, params,
                                      nPatients = nPatients, patient.facLoc = patient.facLoc, patient.facScale = patient.facScale,
                                      nChannels = nChannels, channel.facLoc = channel.facLoc, channel.facScale = channel.facScale) {
    
    print("batch_batch_effects_multi")
    print("in")
    
    nGenes <- getParam(params, "nGenes")
    nBatches <- getParam(params, "nBatches")
    batch.facLoc <- getParam(params, "batch.facLoc")
    batch.facScale <- getParam(params, "batch.facScale")
    means.gene <- rowData(sim)$GeneMean
    
    for (idx in seq_len(nBatches)) {
        batch.facs <- getLNormFactors(nGenes, 1, 0.5, batch.facLoc[idx],
                                        batch.facScale[idx])
        batch.means.gene <- means.gene * batch.facs
        rowData(sim)[[paste0("BatchFacBatch", idx)]] <- batch.facs
    }
    
    patient.facLoc <- rep(patient.facLoc, nPatients*nBatches)
    patient.facScale <- rep(patient.facScale, nPatients*nBatches)
    

    for (idx in seq_len(nPatients*nBatches)) {
        patient.facs <- getLNormFactors(nGenes, 1, 0.5, patient.facLoc[idx],
                                        patient.facScale[idx])
        patient.means.gene <- means.gene * patient.facs
        
        rowData(sim)[[paste0("PatientFacPatient", idx)]] <- patient.facs

    }

    channel.facLoc <- rep(channel.facLoc, nChannels)
    channel.facScale <- rep(channel.facScale, nChannels)
    means.gene <- rowData(sim)$GeneMean

    for (idx in seq_len(nChannels)) {
        channel.facs <- getLNormFactors(nGenes, 1, 0.5, channel.facLoc[idx],
                                        channel.facScale[idx])
        channel.means.gene <- means.gene * channel.facs
        rowData(sim)[[paste0("ChannelFacChannel", idx)]] <- channel.facs
    }
    
    return(sim)
}

In [3]:
splatSimBatchCellMeans_multi <- function(sim, params) {

    print("batch_cell_means_multi")
    print("in")
    nBatches <- getParam(params, "nBatches")
    cell.names <- colData(sim)$Cell
    gene.names <- rowData(sim)$Gene
    gene.means <- rowData(sim)$GeneMean

    if (nBatches > 1) {
        batches <- colData(sim)$Batch
        batch.names <- unique(batches)

        batch.facs.gene <- rowData(sim)[, paste0("BatchFac", batch.names)]

        batch.facs.cell <- as.matrix(batch.facs.gene[,
                                                  as.numeric(factor(batches))])

        patient <- colData(sim)$Patient
        patient.names <- unique(patient)

        ########## here

        patient.facs.gene <- rowData(sim)[, paste0("PatientFac", patient.names)]
        patient.facs.cell <- as.matrix(patient.facs.gene[,
                                                  as.numeric(factor(patient))])

        channel <- colData(sim)$Channel
        channel.names <- unique(channel)

        channel.facs.gene <- rowData(sim)[, paste0("ChannelFac", channel.names)]
        channel.facs.cell <- as.matrix(channel.facs.gene[,
                                                  as.numeric(factor(channel))])

    } else {
        nCells <- getParam(params, "nCells")
        nGenes <- getParam(params, "nGenes")

        batch.facs.cell <- matrix(1, ncol = nCells, nrow = nGenes)
    }
    # Multiplied by all contributing to batch effect
    batch.means.cell <- patient.facs.cell * batch.facs.cell * channel.facs.cell * gene.means 

    colnames(batch.means.cell) <- cell.names
    rownames(batch.means.cell) <- gene.names

    # Captures all
    assays(sim)$BatchCellMeans <- batch.means.cell

    return(sim)
}


In [4]:
splatSimulate_multi_batches <- function(params = newSplatParams(),
                          method = c("single", "groups", "paths"),
                          verbose = TRUE, nChannels = 70,  nPatients = 14, 
                                        channel.facLoc = 0.05, channel.facScale = 0.05, 
                                        patient.facLoc = 0.15, patient.facScale = 0.15, ...) {
    
    print("simulate_multi_batches")
    # nPatients (number of patients per pool)
    checkmate::assertClass(params, "SplatParams")

    method <- match.arg(method)

    if (verbose) {message("Getting parameters...")}
    params <- setParams(params, ...)
    params <- expandParams(params)
    validObject(params)

    # Set random seed
    seed <- getParam(params, "seed")
    set.seed(seed)

    # Get the parameters we are going to use
    nCells <- getParam(params, "nCells")
    nGenes <- getParam(params, "nGenes")
    nBatches <- getParam(params, "nBatches")
    # Used here for pool
    batch.cells <- getParam(params, "batchCells")
    nGroups <- getParam(params, "nGroups")
    group.prob <- getParam(params, "group.prob")

    if (nGroups == 1 && method == "groups") {
        warning("nGroups is 1, switching to single mode")
        method <- "single"
    }

    if (verbose) {message("Creating simulation object...")}
    # Set up name vectors
    cell.names <- paste0("Cell", seq_len(nCells))
    gene.names <- paste0("Gene", seq_len(nGenes))
    batch.names <- paste0("Batch", seq_len(nBatches))
    
    # Names for my extra batches
    patient.names <- paste0("Patient", seq_len(nPatients*nBatches))
    channel.names <- paste0("Channel", seq_len(nChannels))



    if (method == "groups") {
        group.names <- paste0("Group", seq_len(nGroups))
    } else if (method == "paths") {
        group.names <- paste0("Path", seq_len(nGroups))
    }

    # Create SingleCellExperiment to store simulation
    cells <-  data.frame(Cell = cell.names)
    rownames(cells) <- cell.names
    features <- data.frame(Gene = gene.names)
    rownames(features) <- gene.names
    sim <- SingleCellExperiment(rowData = features, colData = cells,
                                metadata = list(Params = params))

    # Make batches vector which is the index of param$batchCells repeated
    # params$batchCells[index] times
    batches <- lapply(seq_len(nBatches), function(i, b) {rep(i, b[i])},
                      b = batch.cells)
    batches <- unlist(batches)
    colData(sim)$Batch <- batch.names[batches]
    
    # Repeat channel number, e.g. 140 channel 1, 140 channel 2, 140 channel 3... etc. 
    channels = rep((1:nChannels), each  = (nCells / nChannels))
    
    # Number of cells in each pool
    length_pool = nCells / nBatches
    
    # Patient label. Set of patients in each pool, e.g. 14 patients in pool1, patients 15-28 in pool 2. Randomised for each cell in that bracket. 
    patients = sapply(1:10, function(x) sample((((x*nPatients)-(nPatients-1)):(x*nPatients)), length_pool ,replace = TRUE)) %>% as.vector()

    # Channel and patient coldata 
    colData(sim)$Patient <- patient.names[patients]
    colData(sim)$Channel <- channel.names[channels]

    if (method != "single") {
        groups <- sample(seq_len(nGroups), nCells, prob = group.prob,
                         replace = TRUE)
        colData(sim)$Group <- factor(group.names[groups], levels = group.names)
    }

    if (verbose) {message("Simulating library sizes...")}
    sim <- splatSimLibSizes(sim, params)
    if (verbose) {message("Simulating gene means...")}
    sim <- splatSimGeneMeans(sim, params)
    if (nBatches > 1) {
        if (verbose) {message("Simulating batch effects...")}
        # Added extra parameter options
        sim <- splatSimBatchEffects(sim, params,
                                   nPatients = nPatients, patient.facLoc = patient.facLoc, patient.facScale = patient.facScale,
                                      nChannels = nChannels, channel.facLoc = channel.facLoc, channel.facScale = channel.facScale)
    }
    sim <- splatSimBatchCellMeans(sim, params)
    if (method == "single") {
        sim <- splatSimSingleCellMeans(sim, params)
    } else if (method == "groups") {
        if (verbose) {message("Simulating group DE...")}
        sim <- splatSimGroupDE(sim, params)
        if (verbose) {message("Simulating cell means...")}
        sim <- splatSimGroupCellMeans(sim, params)
    } else {
        if (verbose) {message("Simulating path endpoints...")}
        sim <- splatSimPathDE(sim, params)
        if (verbose) {message("Simulating path steps...")}
        sim <- splatSimPathCellMeans(sim, params)
    }
    if (verbose) {message("Simulating BCV...")}
    sim <- splatSimBCVMeans(sim, params)
    if (verbose) {message("Simulating counts...")}
    sim <- splatSimTrueCounts(sim, params)
    if (verbose) {message("Simulating dropout (if needed)...")}
    sim <- splatSimDropout(sim, params)

    if (verbose) {message("Done!")}
    return(sim)
}

In [5]:
## Rename in namespace

# Batch effects
environment(splatSimBatchEffects_multi) <- asNamespace('splatter')
assignInNamespace("splatSimBatchEffects", splatSimBatchEffects_multi, ns = "splatter")

# Batch cell means
environment(splatSimBatchCellMeans_multi) <- asNamespace('splatter')
assignInNamespace("splatSimBatchCellMeans", splatSimBatchCellMeans_multi, ns = "splatter")

# Splat simulate multi batches
environment(splatSimulate_multi_batches) <- asNamespace('splatter')
assignInNamespace("splatSimulate", splatSimulate_multi_batches, ns = "splatter")

In [6]:
# Test with random parameters and only 140 cells in each batch
# nPatients = 14
# facLoc and facScale affect batch effect/DE effect quite complexly- mean and standard dev of lognormal distribution, that feed into another function.
test <- splatSimulate_multi_batches(params = newSplatParams(), 
                            batchCells = rep(140,10), method = "groups", group.prob = c(0.2,0.2,0.2,0.2,0.2),
                            verbose = TRUE, nChannels = 70,  nPatients = 14, 
                            channel.facLoc = 0.05, channel.facScale = 0.05, 
                            patient.facLoc = 0.15, patient.facScale = 0.15, 
                            de.prob = c(0.1, 0.1, 0.1, 0.2, 0.2), de.facLoc = 0.2, de.facScale = 0.4,)

[1] "simulate_multi_batches"


Getting parameters...

Creating simulation object...

Simulating library sizes...

Simulating gene means...

Simulating batch effects...



[1] "batch_batch_effects_multi"
[1] "in"
[1] "batch_cell_means_multi"
[1] "in"


Simulating group DE...

Simulating cell means...

Simulating BCV...

Simulating counts...

Simulating dropout (if needed)...

Done!



In [50]:
colData(test)


DataFrame with 1400 rows and 6 columns
                Cell       Batch     Patient     Channel    Group ExpLibSize
         <character> <character> <character> <character> <factor>  <numeric>
Cell1          Cell1      Batch1    Patient1    Channel1   Group4    47198.4
Cell2          Cell2      Batch1    Patient8    Channel1   Group5    55363.7
Cell3          Cell3      Batch1   Patient11    Channel1   Group5    73701.0
Cell4          Cell4      Batch1    Patient3    Channel1   Group4    52904.5
Cell5          Cell5      Batch1    Patient4    Channel1   Group4    57345.7
...              ...         ...         ...         ...      ...        ...
Cell1396    Cell1396     Batch10  Patient127   Channel70   Group1    62644.0
Cell1397    Cell1397     Batch10  Patient131   Channel70   Group1    72766.1
Cell1398    Cell1398     Batch10  Patient138   Channel70   Group2    46221.6
Cell1399    Cell1399     Batch10  Patient130   Channel70   Group4    63424.9
Cell1400    Cell1400     Batch10  Pat

In [ ]:
idx*cells_in_pool - ((idx*cells_in_pool)-1)

In [48]:
nBatches = 10
nCells = 1400
nChannels = 70
cells_in_pool = nCells/nBatches
channels_per_pool = nChannels/nBatches
cells_in_channel = cells_in_pool/channels_per_pool
channel_no = 0 

for(idx in seq(nBatches)){
    
    # Subset large matrix into each pool
    start = (idx*cells_in_pool) - (cells_in_pool - 1) # Start of each pool
    end = idx*cells_in_pool # End of each pool

    sce_subset = test[,start:end]
    cell_barcodes <- sample(barcodes, cells_in_pool, replace = FALSE)
    
    colData(sce_subset)$Cell <- cell_barcodes
    # rownames too
    rownames(colData(test)) <- cell_barcodes
    
    # Each channel per pool
    for(chnl in seq(channels_per_pool)){
        channel_no = channel_no + 1
        start = (chnl*cells_in_channel) - (cells_in_channel - 1)
        end = chnl*cells_in_channel # End of each pool
        
        sce_channel_subset = sce_subset[,start:end]
        
        # Write to file 
        dir <- paste0(c("out_dir/", "channel_", channel_no), collapse = "")
        dir.create(path = dir)

        barcodes_file <- paste0(c(dir, "/quants_mat_rows.txt"), collapse ="" )
        gene_file <- paste0(c(dir, "/quants_mat_cols.txt"), collapse ="" )
        count_file <- paste0(c(dir, "/quants_mat.csv"), collapse ="" )

        write.table(rownames(sce_channel_subset), file= gene_file, quote=FALSE, col.names=FALSE, row.names=FALSE)
        write.table(colnames(sce_channel_subset), file= barcodes_file, quote=FALSE, col.names=FALSE, row.names=FALSE)
        write.table(counts(sce_channel_subset), file= count_file, quote=FALSE, col.names=FALSE, row.names=FALSE, sep=",") 
        
    }
    
    
    
}

[1] 1
[1] 140
[1] 141
[1] 280
[1] 281
[1] 420
[1] 421
[1] 560
[1] 561
[1] 700
[1] 701
[1] 840
[1] 841
[1] 980
[1] 981
[1] 1120
[1] 1121
[1] 1260
[1] 1261
[1] 1400


In [35]:
colData(test)$Channel == "Channel1"

[1]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
  [13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE
  [25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [73] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [85] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [145] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [157] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [169] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [181] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [193] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [205] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [217] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [229] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [253] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [265] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [277] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [289] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [301] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [313] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [337] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [361] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [397] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [433] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [445] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [457] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [469] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [481] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [493] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [505] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [517] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [529] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [541] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [553] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [565] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [577] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [589] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [601] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALS

In [14]:
barcodes <- read_delim("737K-august-2016.txt", delim = "\t", col_names =  FALSE)

Parsed with column specification:
cols(
  X1 = col_character()
)



In [18]:
barcodes <- barcodes$X1

In [26]:
colData(test)$Cell %>% length
length(barcodes)
sample(barcodes,(colData(test)$Cell %>% length), replace = FALSE ) 

# barcodes[1:(colData(test)$Cell %>% length)]

[1] 1400

[1] 737280

[1] "GCGCCAACAGGTCTCG" "TGAGCCGAGTAGGCCA" "CCACTACAGATCTGAA"
   [4] "CTTTGCGGTTTAGCTG" "CTCGAAAGTACCGGCT" "CAGCAGCCAAATACAG"
   [7] "CGCCAAGCATGGTAGG" "GGAACTTTCACAGTAC" "ACGCCAGAGCTGGAAC"
  [10] "GCTGCTTAGAAGGACA" "TCAGCAAGTCAACTGT" "CATGGCGCACGAAACG"
  [13] "AACTCAGCAAACGCGA" "ATTGGACGTACGCACC" "CTTCTCTCATGTAAGA"
  [16] "CGCGGTAGTGCAGACA" "CTCGAAAAGACGCAAC" "TACCTATAGCTAGTGG"
  [19] "CGGAGTCAGATATACG" "ATCCGAATCACATGCA" "TTGTAGGGTCGAAAGC"
  [22] "AGCGTATTCATTGCGA" "GCATGCGAGATGAGAG" "AGGTCCGAGTGTCCAT"
  [25] "TATTACCAGAATTCCC" "CGTCACTTCTAACTCT" "TGGCTGGGTATTACCG"
  [28] "AGGTCCGCATTGGTAC" "CCTTCGAAGCCACGCT" "CGTCACTGTATCAGTC"
  [31] "TCAGGTACACGGCTAC" "TGTGGTACATATGAGA" "ACGATGTAGGCCCTTG"
  [34] "TGCTACCGTATTCGTG" "CACACCTCATACGCCG" "CAAGTTGAGTGCGATG"
  [37] "AAGACCTCAACGATCT" "CGCGGTAGTCTCATCC" "TGAGCCGAGACAATAC"
  [40] "TAGCCGGAGGATGGAA" "TGACTAGTCTATCCCG" "TGCTGCTCACGGTAAG"
  [43] "GTCTTCGAGTGGAGAA" "CAACCAACAGCTGCAC" "GCTCCTAGTACACCGC"
  [46] "CGACCTTGTCAGAAGC" "GGCAATTTCTGTTGAG" "CTCTACGAGTAGATGT"
  [49] "CAGTAACTCAGAGCTT" "CAGCGACAGCGTTCCG" "GTCGGGTCATGGTCAT"
  [52] "GGTGCGTAGGTAGCCA" "AGCTCCTGTGCCTTGG" "GACAGAGCACACGCTG"
  [55] "GTTCATTAGGGTTTCT" "CCAGCGAGTATATGAG" "AAAGTAGAGTGGAGTC"
  [58] "TACCTTAAGCTAGGCA" "TATTACCTCGAACGGA" "CCAGCGAGTACCGGCT"
  [61] "TACGGGCTCCGTAGGC" "CCGGTAGAGGGTCTCC" "GCTGCAGGTGTTTGTG"
  [64] "AACACGTGTTCAACCA" "ACTTTCAAGAAACCTA" "TGCGCAGCACAGCCCA"
  [67] "GGTGTTATCGCCGTGA" "ACGATGTTCAGTGCAT" "CCGGGATCAGGCTCAC"
  [70] "CAAGGCCCAGTAAGCG" "AGCGGTCAGGCCCTCA" "TGACTTTGTGATGCCC"
  [73] "CATCGAAGTTGGAGGT" "TAAGCGTGTTCACGGC" "AGGTCATAGCCAGTTT"
  [76] "AACTCTTCATTGAGCT" "TCTCTAAGTAAGGGCT" "ATTACTCTCACCGTAA"
  [79] "CCTTCCCCATCGGACC" "CATATGGTCCGCAGTG" "ACATGGTCATCTCCCA"
  [82] "ACACCAAGTTAAGATG" "CTGTGCTCATCCAACA" "ACGAGCCGTAACGTTC"
  [85] "GATTCAGCATGTCCTC" "TTCTACACAGCTATTG" "TCAACGAAGCTACCTA"
  [88] "GCATGTATCTTTACAC" "GCTGCGACACTGTGTA" "GCGACCATCGAATCCA"
  [91] "CTACACCAGCTGCCCA" "CCTTCCCAGTAGATGT" "CTCACACCAGAGTGTG"
  [94] "TTAACTCCAAGTTAAG" "TACCTTAAGAGGGCTT" "CCCTCCTCATTACGAC"
  [97] "GATGCTATCGCCTGAG" "GTAGGCCTCAGATAAG" "GTACTCCGTTCATGGT"
 [100] "TCAATCTGTTCCACTC" "ATGCGATCAGCTGCTG" "CACAGTAAGTCCGGTC"
 [103] "GACTACAAGCCTATGT" "AGACGTTTCCCTTGCA" "TTGTAGGGTCCGAATT"
 [106] "TGAGGGACAGGTCGTC" "TACCTATCAGCTTAAC" "AAGGTTCGTGCCTGTG"
 [109] "CTACCCATCGTTGACA" "CATGGCGCACGCTTTC" "CCGTTCAAGTTGCAGG"
 [112] "CTACGTCCAAGTAATG" "AACTCCCAGTGTCCAT" "CGTCAGGAGTCGCCGT"
 [115] "CTTGGCTTCGTGTAGT" "GCGCCAAAGGTGGGTT" "TAAACCGGTAGATTAG"
 [118] "CGCGGTACAGTAGAGC" "AAAGCAATCTGATTCT" "GTACGTAAGTACGACG"
 [121] "GTCTCGTCACACCGCA" "CAGCCGATCTGGCGAC" "ACTGATGCATCGATTG"
 [124] "TTTGTCACATGCTGGC" "GTCACGGAGAGCCCAA" "TAGGCATCAGAGCCAA"
 [127] "TCAGGATCATGCCTAA" "TGGCCAGAGTCGCCGT" "TTGCGTCAGATCCCGC"
 [130] "GAAGCAGCATGCCTTC" "TTGCGTCTCCACGCAG" "CGTCTACGTCAACTGT"
 [133] "CAGAATCCAGACGTAG" "GTTACAGCAAGCCTAT" "TCCCGATAGTACTTGC"
 [136] "ACCCACTTCAACACCA" "CCGGTAGAGGGATCTG" "TTGGAACCAGTACACT"
 [139] "TCTTTCCGTAGCGATG" "GCGCAGTTCGCGATCG" "GAGTCCGAGAGGACGG"
 [142] "GCTTGAAAGTTCGATC" "CGAATGTAGGCTAGAC" "AGGGTGATCGAATGGG"
 [145] "TCGAGGCCAGACAAAT" "TCGGTAACAATCCAAC" "AACGTTGGTAAGAGAG"
 [148] "CCGTTCATCTGCGGCA" "CTCAGAAAGGTGCTAG" "TCTGGAACATGGGACA"
 [151] "CCACGGACAGGAACGT" "GCGCGATAGTGGGTTG" "CAGGTGCGTATGAATG"
 [154] "CCCAATCGTCTGGAGA" "TGGTTAGCACATTTCT" "CCACCTAAGAAGGACA"
 [157] "TGGTTCCGTCAAGCGA" "TGGTTCCTCAAGCCTA" "CCACTACGTAAGTGTA"
 [160] "TTTGTCACATATGAGA" "GACGTTAGTACAAGTA" "CGTAGCGGTTCAACCA"
 [163] "TCGTAGATCCTCAACC" "TGCGGGTTCTCCGGTT" "TTGGAACAGATGCCAG"
 [166] "AATCGGTGTCCTCCAT" "CTTAACTAGCGCTCCA" "GTTACAGAGATGGGTC"
 [169] "GAGGTGAGTAGCCTCG" "CATCAGAGTGCGCTTG" "ACAGCTAGTTAGGGTG"
 [172] "CATCGGGCAGTATAAG" "AACTCCCCACCATGTA" "TACTCGCCATTGGTAC"
 [175] "AGCTTGAAGGACACCA" "CTTGGCTCATAACCTG" "TAGCCGGAGACGCACA"
 [178] "GGAACTTTCATCGCTC" "CTCGGAGAGGTGACCA" "GGAACTTTCCGCAAGC"
 [181] "GTACTTTCAGACAGGT" "CGGTTAAAGAGGTTAT" "GATCGTACACAGGTTT"
 [184] "TCTTCGGAGTGTCTCA" "CAGTCCTAGATCACGG" "CAGCTGGCACGGTAAG"
 [187] "ACCTTTAAGTCATGCT" "TATCTCAT